# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [36]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score #easly calculates percentile score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [37]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [38]:
symbol = 'AAPL'
api_url=f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data



{'companyName': 'Apple Inc',
 'marketcap': 2295321807940,
 'week52high': 185.57,
 'week52low': 134.03,
 'week52highSplitAdjustOnly': 187.27,
 'week52lowSplitAdjustOnly': 130.54,
 'week52change': 0.02017894578730422,
 'sharesOutstanding': 16591087995,
 'float': 0,
 'avg10Volume': 104021299,
 'avg30Volume': 90699707,
 'day200MovingAvg': 165.49,
 'day50MovingAvg': 144.75,
 'employees': 149938,
 'ttmEPS': 6.32,
 'ttmDividendRate': 0.918268724168624,
 'dividendYield': 0.006776776087562225,
 'nextDividendDate': '',
 'exDividendDate': '2022-04-26',
 'nextEarningsDate': '2022-07-24',
 'peRatio': 22.448110322205032,
 'beta': 1.3171032277775532,
 'maxChangePercent': 53.50074829653322,
 'year5ChangePercent': 2.942900571553133,
 'year2ChangePercent': 0.5758685841743013,
 'year1ChangePercent': 0.03286493037427344,
 'ytdChangePercent': -0.2438955644001581,
 'month6ChangePercent': -0.2212592307779688,
 'month3ChangePercent': -0.1803850143925221,
 'month1ChangePercent': -0.012903434219194942,
 'day30C

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [39]:
data['year1ChangePercent']

0.03286493037427344

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [40]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
#print(len(symbol_groups))
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    #print(symbol_strings[i])


my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [41]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=price,stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['price'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
            
        ],
            index = my_columns
        ),
            ignore_index=True
        )
        
final_dataframe

KeyError: 'DISCA'

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,42,APA,39.300,0.861219,N/A
1,89,CF,89.000,0.807013,N/A
2,109,COP,93.060,0.713282,N/A
3,123,CVX,154.800,0.467465,N/A
4,54,AZO,2129.140,0.452743,N/A
5,23,ALB,214.370,0.353041,N/A
6,107,COG,23.360,0.347208,N/A
7,66,BKR,29.471,0.320342,N/A
8,11,ADM,80.290,0.3125,N/A
9,4,ABBV,153.189,0.31154,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [42]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your porfolio: ')
    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number /nPlease try again!')
        portfolio_size = input('Enter the size of your porfolio: ')
        
portfolio_input()
print(portfolio_size)

Enter the size of your porfolio: 10000000
10000000


In [43]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])

    
final_dataframe
    

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,117.630,-0.225524,629
1,AAL,13.292,-0.433769,5572
2,AAP,174.680,-0.115614,424
3,AAPL,141.230,0.033648,524
4,ABBV,147.490,0.304437,502
...,...,...,...,...
130,DG,242.770,0.131763,305
131,DGX,135.572,0.029241,546
132,DHI,66.000,-0.323584,1122
133,DHR,251.660,-0.070407,294


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [73]:
hqm_columns = [
    'Ticker',
    'Price',
    'Numbe of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)
#print(symbol_strings)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=price,stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['price'],
            'N/A',
            data[symbol]['stats']['year1ChangePercent'],
            'N/A',
            data[symbol]['stats']['month6ChangePercent'],
            'N/A',
            data[symbol]['stats']['month3ChangePercent'],
            'N/A',
            data[symbol]['stats']['month1ChangePercent'],
            'N/A',
            'N/A'
        ],
            index=hqm_columns,
        ),
            ignore_index=True
        )
        
hqm_dataframe

KeyError: 'DISCA'

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [74]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100
        
hqm_dataframe

TypeError: '<' not supported between instances of 'NoneType' and 'float'

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [75]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentile = []
    for time_period in time_periods:
        momentum_percentile.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentile)
hqm_dataframe

TypeError: can't convert type 'str' to numerator/denominator

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [81]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True, drop=True)

hqm_dataframe

KeyError: 'HQM Score'

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [82]:
portfolio_input()

Enter the size of your porfolio: 100000000


In [83]:
position_size = float(portfolio_size)/len(hqm_dataframe)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Numbers of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])

hqm_dataframe




,Ticker,Price,Numbe of Shares to Buy,One-Year Price Return,Six-Month Price Return,Three-Month Price Return,One-Month Price Return,Numbers of Shares to Buy
0,A,116.04,N/A,-0.222965,-0.264723,-0.181473,-0.093715,17235
1,CDNS,150.253,N/A,0.122389,-0.196671,-0.058064,0.010968,13310
2,CMCSA,39.88,N/A,-0.319909,-0.206295,-0.178953,-0.085141,50150
3,CMA,75.91,N/A,0.112936,-0.093309,-0.195206,0.049939,26346
4,CLX,130.18,N/A,-0.262447,-0.261751,-0.056114,-0.09227,15363
5,CL,80.247,N/A,-0.040083,-0.058549,0.038496,0.017971,24923
6,CINF,118.19,N/A,0.011082,0.048656,-0.116768,-0.04837,16921
7,CI,264.46,N/A,0.082524,0.155903,0.041184,-0.023987,7562
8,CHTR,463.09,N/A,-0.387051,-0.322386,-0.228105,-0.087066,4318
9,CHRW,100.0,N/A,0.10268,0.005819,-0.034783,-0.001558,20000


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [66]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Startegy', index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [67]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [68]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Numbe of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template ],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Startegy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Startegy'].write(f'{column}1', column_formats[column][0], string_template)



## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [69]:
writer.save()